# **Proyecto IA**

Juan David Cetina, Mariana Salas, Santiago Sabogal

## **1. Introducción**

En este proyecto se desarrolló, entrenó y evaluó un agente para el juego de Connect4 utilizando técnicas de aprendizaje por refuerzo. El objetivo principal fue construir una política capaz de tomar decisiones racionales a partir de la experiencia, empleando una Q-table aprendida mediante Q-learning tabular. Para ello, se implementó un entorno de juego completo y se diseñó un proceso de entrenamiento donde el agente juega 12,000 partidas contra un oponente heurístico. Durante este proceso, el agente aplica una estrategia UCB para equilibrar exploración y explotación, actualiza sus Q-values después de cada episodio y genera un archivo de modelo (*connect4_model.json*) que recoge el conocimiento adquirido.

Posteriormente, se integró esta Q-table en una policy final que combina tres elementos: búsqueda táctica de victorias y bloqueos inmediatos, consulta de los valores aprendidos, y una heurística basada en la centralidad del tablero para resolver estados no vistos. Asimismo, se generaron visualizaciones y métricas que permiten evaluar la evolución del agente, como curvas de recompensa, longitudes de partida, tasas de victoria frente al oponente heurístico y estadísticas completas de visitas y Q-values. Finalmente, el modelo entrenado fue utilizado para validar que la policy se vuelve progresivamente más racional y consistente, mostrando un comportamiento coherente con la dinámica del aprendizaje reforzado.

## **2. Metodología**

La metodología aplicada en este proyecto integra conceptos teóricos de toma de decisiones secuenciales, procesos de decisión de Markov y aprendizaje por refuerzo. El objetivo es desarrollar un agente para Connect4 que combine aprendizaje tabular con heurísticas tácticas para lograr un comportamiento útil y competitivo.

### **2.1. Propuestas iniciales**

#### **2.1.1. Agente heurístico**

Inicialmente, se hizo un agente heurístico en *policy_heuristica.py*, utiliza búsqueda minimax con poda alfa–beta y una evaluación heurística del tablero en lugar de MCTS. A partir del tablero actual, el agente identifica primero de quién es el turno y genera las columnas legales. Luego, aplica dos tácticas inmediatas, si existe una jugada ganadora la ejecuta, y si el rival puede ganar en una jugada, la bloquea. 

Cuando no hay táctica directa, se explora el árbol de juego hasta una profundidad fija usando minimax. Los nodos donde juega el agente intentan maximizar una función de evaluación, y los nodos del rival intentan minimizarla, mientras la poda alfa–beta descarta ramas que no pueden mejorar el resultado. La evaluación heurística recorre todas las “ventanas” posibles de 4 celdas (horizontales, verticales y diagonales) y asigna puntuaciones en función de alineaciones prometedoras del agente y amenazas del oponente. También, da un peso extra al control de la columna central. Entonces, los estados terminales se refuerzan con valores muy altos (victoria) o muy bajos (derrota), de forma que la búsqueda prioriza líneas que llevan a ganar y evita las que llevan a perder.

Esta fue la entrega del avance. Sin embargo, en la retroalimentación se explicó que los movimientos predeterminadas, pese a no estar mal y mostrar un buen rendimiento, siguen siendo principalmente un baseline. No hay aprendizaje ni verdadera racionalidad, son solo reglas específicas de juego con cierta profundidad limitada en la anticipación de decisiones futuras.

#### **2.1.2. Agente Monte Carlo Tree Search**

Siguendo las recomendaciones del profesor, se cambió el enfoque heurístico a uno de aprendizaje. En este, se tiene un agente para Connect4 llamado *MctsUcbPolicy* que decide cada jugada combinando búsqueda Monte Carlo Tree Search (MCTS) con la regla de selección UCB1 y una tabla de valores previa aprendida por self-play. En cada llamada a act, el agente primero aplica una táctica inmediata, si puede ganar en una jugada lo hace, y si el rival puede ganar en una jugada, bloquea esa amenaza. Si ninguna de estas condiciones aplica, se construye un árbol de búsqueda a partir del estado actual y se ejecuta un número fijo de simulaciones. En cada simulación se realiza el ciclo de MCTS (selección mediante UCB, expansión de nodos nuevos, simulación con una política ligera sesgada al centro del tablero y retropropagación de las recompensas). Además, se incorpora una tabla *values.json* donde se guardan valores aproximados V(s) obtenidos en el entrenamiento, que se utilizan como prior suave inicializando algunos nodos con visitas y recompensas virtuales. Entonces, el árbol se guía hacia estados que ya han demostrado ser prometedores. Al finalizar las simulaciones, el agente selecciona como acción final la columna asociada al hijo de la raíz con mayor número de visitas, favoreciendo en caso de empate a la columna central.

Sin embargo, pese a que fue entrenado miles de veces, no lograba pasar la simulación en Gradescope para ganarle a agentes aleatorios, así empezará como el jugador rojo o siendo el jugador amarillo.

Este agente puede encontrarse en *policy_mcts.py*.

#### **2.1.3. Agente con política de entrenamiento ε-Greedy**

Teniendo en cuenta lo mencionado previamente, se hace un agente con aprendizaje por refuerzo tabular para Connect4. Se tiene la policy *CetinaSalasSabogal* y un código de entrenamiento Q-learning contra un oponente heurístico (*train_heuristic.py*). La policy decide su jugada en tres escenarios. Primero, busca una victoria inmediata simulando cada columna disponible. Luego, intenta bloquear una posible victoria del rival en el siguiente turno y, si ninguna de esas tácticas aplica, consulta una Q-table aprendida, cargada desde *connect4_model.json* para elegir la acción legal con mayor valor Q. Entonces, si el estado no está en la tabla, recurre a una heurística fija que prioriza la columna central y las cercanas al centro. El script de entrenamiento, por su parte, entrena esa Q-table jugando miles de episodios contra un oponente, que también sabe ganar y bloquear en una jugada y prefiere el centro con la finalidad de competir contra un oponente más fuerte que uno random para el torneo. Este usa Q-learning con política ε-greedy (explora al inicio y se va volviendo más greedy con el tiempo), actualizando Q(s,a) al final de cada partida.

Este demuestra una curva de aprendizaje y logra pasar la prueba de Gradescope. Sin embargo, no fue escogida como la propuesta final debido a que presenta un desempeño menor a la seleccionada.

### **2.2. MDP**

Para el trabajo final, el juego Conenect4 se formuló como un Proceso de Decisión de Markov (MDP):

- **Estados \( S \):** Todas las configuraciones del tablero de 6 X 7, junto con el jugador al que le corresponde mover.
- **Acciones \( A \):** Columnas legales donde puede colocarse una ficha.
- **Transiciones \( P(s'| s, a) \):** Deterministas, porque una jugada siempre produce el mismo estado.
- **Recompensas \( r(s) \):**
  - \( +1 \) si el agente gana.
  - \( -1 \) si pierde.
  - \( 0 \) en empates.
- **Factor de descuento:** 0.95.

A continuación, se muestran fórmulas importantes a tener en cuenta.

$$
v^\pi(s) = \mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t \, r(S_t)\right]
$$

$$
q^\pi(s,a) = r(s) + \gamma \sum_{s'} P(s' \mid s,a)\, v^\pi(s')
$$

### **2.3. Política final**

La política final implementada en *CetinaSalasSabogal*, en *policy.py*, combina dos componentes principales:  
* **Heurística táctica**: Diseñada para detectar oportunidades de victoria y amenazas inmediatas. 
* **Q-table:** Aprendida durante el entrenamiento para seleccionar acciones con base en los valores Q almacenados.  

Este enfoque sigue el esquema de General Policy Iteration (GPI), ya que la política mejora mediante valores aprendidos, pero recurre a reglas tácticas cuando no posee información suficiente para un estado. Antes de cada partida, el método *mount()* intenta cargar, si existe, el archivo con el aprendizaje: *connect4_model.json*. Si el archivo no existe, contiene errores o está vacío, la Q-table se inicializa y la política opera únicamente con heurística.

Cuando sí existe información para el estado actual, la política selecciona:

$$
\pi(s) = \arg\max_{a \in A(s)} Q(s,a)
$$

El proceso es:

1. Codificar el estado con *codificar_posicion(tablero)*.  
2. Recuperar *self._q[estado]* si existe.  
3. Filtrar solo las acciones legales.  
4. Elegir la columna con mayor Q-valor.

Cuando no hay Q-valores útiles para el estado actual, la policy usa una heurística determinista compuesta por tres pasos:

* Buscar victoria inmediata, donde para cada columna legal se simula la jugada (*simular_caida*), se evalúa si produce un cuatro en línea usando *cuatro_en_linea()* y, si es victoria, juega esa columna inmediatamente.

* Bloquear victoria del rival, donde si el rival puede ganar en su próximo turno se identifica la columna peligrosa y se juega ahí para bloquear y evitar la derrota.

* Prioridad posicional, ya que si no hay ataques ni amenazas inmediatas, la política aplica esta prioridad de:
    1. Centro  
    2. Columnas adyacentes  
    3. Columnas intermedias  
    4. Esquinas

### **2.4. Política de entrenamiento**

Para el entrenamiento se implementa un esquema de aprendizaje por refuerzo en el que el agente aprende una tabla Q sobre el juego Conecta 4, con una estrategia de Upper Confidence Bound (UCB). El objetivo es equilibrar la explotación (usar las acciones con mejor valor estimado) y exploración (probar acciones de las que se tiene menos información). El entorno se modela como un MDP, como se mencionó anteriormente.

El entrenamiento utiliza un oponente determinista, que actúa como baseline y permite evaluar si el agente aprende a superar otras estrategias. Este oponente replica una política heurística de Connect4 basada en las siguientes reglas tácticas:

1. **Victoria inmediata:** Si existe una jugada ganadora en el turno actual, la ejecuta.
2. **Bloqueo inmediato:** Si el rival tiene una jugada ganadora en su siguiente turno, la bloquea.
3. **Prioridad posicional:** Prefiere jugar en columnas centrales (mejor control y más combinaciones posibles).
4. **Evitar derrotas inmediatas:** Identifica jugadas que le darían al rival una victoria automática y las evita cuando es posible.

Esta política garantiza que el agente se entrene contra un oponente consistente, lo que permite al algoritmo Q-learning aprender patrones de ataque y defensa más complejos.

Acerca del UCB, este se implementa mediante la fórmula:

$$
\text{UCB}(s,a) \;=\; Q(s,a) \;+\; c \sqrt{\frac{\ln N(s)}{N(s,a)}}
$$

- **\(Q(s,a)\):** Es el valor aprendido para la acción.
- **\(N(s,a)\):** Es el número de veces que se ha elegido la acción \(a\) en el estado \(s\).
- **\(N(s)\):** Es el número total de visitas a ese estado.
- **\(c\):** Es el parámetro de exploración (c = 1.4), siguiendo el recomendado en las diapositivas de clase.

Siguiendo esta estrategia, la política exploratoria del agente queda definida como:

$$
\pi_{\text{UCB}}(s) \;=\; 
\arg\max_{a \in A(s)} \left[ Q(s,a) + c \sqrt{\frac{\ln N(s)}{N(s,a)}} \right]
$$

Esta regla de decisión permite que el agente explore de forma inteligente y adaptativa, priorizando acciones poco investigadas. Los valores generados se van actualizando en la tabla.